# Chapter 04: 도구 사용 (Tool Use)

이 노트북에서는 LLM에 도구를 바인딩하고 활용하는 방법을 학습합니다.

## 주요 내용
- 계산기 도구 정의 및 사용
- Wikipedia 검색 도구 통합
- 도구 호출 결과 처리

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TeeDDub/Building-Applications-with-AI-Agents/blob/main/notebook/ch04_tool_use.ipynb)


## 1. 패키지 설치


In [ ]:
!pip install -q langchain langchain-openai langchain-community langgraph langchain-mcp-adapters openai wikipedia fastapi pydantic uvicorn python-dotenv


## 2. API 키 설정


In [ ]:
import os

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Colab Secrets에서 API 키를 불러왔습니다.")
except:
    pass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"
    print("⚠️ API 키를 직접 입력해주세요.")


## 3. calculator_tool_use.py


기본 산술 도구를 정의하고 LLM에 바인딩합니다.


In [ ]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

# 도구 정의
@tool
def multiply(x: float, y: float) -> float:
    """'x'와 'y'를 곱합니다."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """'x'를 'y'제곱합니다."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """'x'와 'y'를 더합니다."""
    return x + y

tools = [multiply, exponentiate, add]

# LLM 초기화 및 도구 바인딩
llm = init_chat_model(model="gpt-5-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

query = "393 * 12.25는 얼마인가요? 그리고 11 + 49는요?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {
        "add": add,
        "multiply": multiply,
        "exponentiate": exponentiate
    }[tool_call["name"]]
    result = selected_tool.invoke(tool_call['args'])
    
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
    print(f"Result: {result}")
    print()
    
    # ToolMessage 생성
    tool_msg = ToolMessage(content=str(result), tool_call_id=tool_call["id"])
    messages.append(tool_msg)

final_response = llm_with_tools.invoke(messages)
print(final_response.content)


## 4. wikipedia_tool_use.py


Wikipedia 검색 도구를 통합해 외부 지식을 조회합니다.


In [ ]:
from langchain.chat_models import init_chat_model
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.messages import HumanMessage

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# LLM 초기화 및 도구 바인딩
llm = init_chat_model(model="gpt-5-mini", temperature=0)
llm_with_tools = llm.bind_tools([tool])

messages = [HumanMessage("Buzz Aldrin의 주요 업적은 무엇인가요?")]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    tool_msg = tool.invoke(tool_call)
    
    print(tool_msg.name)
    print(tool_call['args'])
    print(tool_msg.content)
    messages.append(tool_msg)
    print()

final_response = llm_with_tools.invoke(messages)
print(final_response.content)


## 5. stock_price_tool_use.py


주식 가격 조회 도구 예제를 구성합니다.


In [ ]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
import requests

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

@tool
def get_stock_price(ticker: str) -> float:
    """주식 시장 거래소 거래 티커에 대한 주식 가격을 가져옵니다."""
    api_url = f"https://api.example.com/stocks/{ticker}"
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            return response.json()["price"]
        else:
            return f"주식 가격을 가져오는데 실패했습니다: {ticker}"
    except requests.exceptions.RequestException:
        return f"주식 가격을 가져오는데 실패했습니다: {ticker}"


# LLM 초기화 및 도구 바인딩
llm = init_chat_model(model="gpt-5-mini", temperature=0)
llm_with_tools = llm.bind_tools([get_stock_price])

messages = [HumanMessage("애플의 현재 주식 가격은 얼마인가요?")]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    tool_msg = get_stock_price.invoke(tool_call)
    
    print(tool_msg.name)
    print(tool_call['args'])
    print(tool_msg.content)
    messages.append(tool_msg)
    print()

final_response = llm_with_tools.invoke(messages)
print(final_response.content)


## 6. pokemon_type_tool_use.py


포켓몬 타입 조회 도구 예제를 구성합니다.


In [ ]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
import requests

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

@tool
def get_pokemon_type(pokemon: str) -> str:
    """포켓몬의 타입을 가져옵니다."""
    api_url = f"https://pokeapi.co/api/v2/pokemon/{pokemon.lower()}"
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            types = [t["type"]["name"] for t in data["types"]]
            return ", ".join(types)
        else:
            return f"포켓몬의 타입을 가져오는데 실패했습니다: {pokemon}"
    except requests.exceptions.RequestException:
        return f"포켓몬의 타입을 가져오는데 실패했습니다: {pokemon}"


# LLM 초기화 및 도구 바인딩
llm = init_chat_model(model="gpt-5-mini", temperature=0)
llm_with_tools = llm.bind_tools([get_pokemon_type])

messages = [HumanMessage("피카츄의 타입은 무엇인가요?")]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    tool_msg = get_pokemon_type.invoke(tool_call)
    
    print(tool_msg.name)
    print(tool_call['args'])
    print(tool_msg.content)
    messages.append(tool_msg)
    print()

final_response = llm_with_tools.invoke(messages)
print(final_response.content)


## 7. langgraph_mcp_client.py


이 코드는 MCP 서버와의 연결이 필요합니다. Colab에서 바로 실행하기 어려우니 원본 파일과 MCP 서버 실행 절차를 참고하세요.


LangGraph 기반 MCP 클라이언트를 구성합니다.


In [ ]:
import asyncio
from typing import Any, Sequence, TypedDict

from langchain_core.messages import HumanMessage
from langchain_core.tools import Tool
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph


class AgentState(TypedDict):
    messages: Sequence[Any]


# MCP 도구 클라이언트 생성
mcp_client = MultiServerMCPClient(
    {
        "math": {
            "command": "python3",
            "args": ["ch04/mcp_servers/MCP_math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # `python ch04/mcp/MCP_weather_server.py`으로 MCP 서버를 먼저 실행합니다.
            "url": "http://0.0.0.0:8000/mcp",
            "transport": "streamable_http",
        },
    }
)

# MCP 도구 클라이언트에서 도구 목록 가져오기
async def get_mcp_tools() -> list[Tool]:
    return await mcp_client.get_tools()


async def call_mcp_tools(state: AgentState) -> dict[str, Any]:
    """
    state["messages"]를 포함한 AgentState를 받아 어떤 MCP 도구를 호출할지 결정합니다.
    데모 목적으로, 사용자 메시지에 'weather'나 '날씨'가 포함되어 있으면 날씨 도구를 호출하고,
    수학 표현식(숫자/연산자)이 포함되어 있으면 수학 도구를 호출합니다.
    그 외의 경우에는 기본 메시지를 반환합니다.
    """
    messages = state["messages"]
    last_msg = messages[-1].content.lower()

    # MCP_TOOLS를 전역 변수로 선언하여 한 번만 가져옵니다.
    global MCP_TOOLS
    if "MCP_TOOLS" not in globals():
        MCP_TOOLS = await mcp_client.get_tools()

    # 간단한 판단: 숫자/연산자 토큰이 있으면 "math"를, 'weather'나 '날씨'가 포함되어 있으면 "weather"를 선택합니다.
    # 실제로는 LLM이 적절한 도구를 판단하지만 여기서는 시연을 위해 간단하게 판단합니다.
    if any(token in last_msg for token in ["+", "-", "*", "/", "(", ")"]):
        tool_name = "math"
        tool_input = {"expression": messages[-1].content}
    elif "weather" in last_msg or "날씨" in messages[-1].content:
        tool_name = "weather"
        content = messages[-1].content

        if "weather in" in content.lower():
            location = content.lower().split("weather in")[1].strip().rstrip("?").strip()
        elif "의 날씨" in content:
            location = content.split("의 날씨")[0].strip()
        else:
            location = "NYC"
        tool_input = {"location": location}
    else:
        return {
            "messages": [
                {"role": "assistant", "content": "수학 또는 날씨 질문만 답변할 수 있습니다."}
            ]
        }

    tool_obj = next((t for t in MCP_TOOLS if t.name == tool_name), None)
    if tool_obj is None:
        return {
            "messages": [
                {"role": "assistant", "content": f"{tool_name} 도구를 사용할 수 없습니다."}
            ]
        }

    mcp_result: str = await tool_obj.ainvoke(tool_input)

    return {"messages": [{"role": "assistant", "content": mcp_result}]}


def construct_graph():
    g = StateGraph(AgentState)
    g.add_node("assistant", call_mcp_tools)
    g.set_entry_point("assistant")
    return g.compile()


GRAPH = construct_graph()


async def run_math_query():
    initial_state = {"messages": [HumanMessage(content="(3 + 5) * 12은 얼마인가요?")]}
    result = await GRAPH.ainvoke(initial_state)
    assistant_msg = result["messages"][-1]
    content = (
        assistant_msg.get("content")
        if isinstance(assistant_msg, dict)
        else assistant_msg.content
    )
    print("Math answer:", content)


async def run_weather_query():
    initial_state = {"messages": [HumanMessage(content="NYC의 날씨는 어때요?")]}
    result = await GRAPH.ainvoke(initial_state)
    assistant_msg = result["messages"][-1]
    print("Weather answer:", assistant_msg["content"])


if __name__ == "__main__":
    asyncio.run(run_math_query())
    asyncio.run(run_weather_query())


## 8. MCP_math_server.py


이 코드는 MCP 서버를 실행합니다. Colab에서는 서버 실행이 제한될 수 있으니 원본 파일을 참고해 로컬에서 실행하세요.


MCP 수학 도구 서버를 정의합니다.


In [ ]:
#!/usr/bin/env python3
"""
표준 MCP (Model Context Protocol) Math Server
JSON-RPC 2.0 프로토콜을 사용하여 수학 계산을 수행합니다.
"""
import sys
import json
import ast
import operator
from typing import Any, Dict

# ─── Safe Expression Evaluation ────────────────────────────────────────────────
ALLOWED_OPERATORS = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Pow: operator.pow,
    ast.USub: operator.neg,
}

def eval_expr(node: ast.AST) -> float:
    # Python 3.8+ 에서는 ast.Constant를 사용 (ast.Num은 Python 3.14에서 제거됨)
    if isinstance(node, ast.Constant):
        return node.value
    if isinstance(node, ast.BinOp) and type(node.op) in ALLOWED_OPERATORS:
        left = eval_expr(node.left)
        right = eval_expr(node.right)
        return ALLOWED_OPERATORS[type(node.op)](left, right)
    if isinstance(node, ast.UnaryOp) and type(node.op) in ALLOWED_OPERATORS:
        operand = eval_expr(node.operand)
        return ALLOWED_OPERATORS[type(node.op)](operand)
    raise ValueError(f"Unsupported expression: {ast.dump(node)}")

def compute_math(expression: str) -> float:
    """안전하게 산술 표현식을 파싱하고 평가합니다."""
    try:
        # 수식만 추출 (숫자와 연산자만)
        cleaned = "".join(ch for ch in expression if ch.isdigit() or ch in "+-*/()^ .*")
        cleaned = cleaned.replace("^", "**").strip()
        if not cleaned:
            raise ValueError("수식을 찾을 수 없습니다.")
        
        expr_ast = ast.parse(cleaned, mode="eval").body
        return eval_expr(expr_ast)
    except Exception as e:
        raise ValueError(f"수식 '{expression}' 계산 오류: {e}")

# ─── JSON-RPC 2.0 Handler ───────────────────────────────────────────────────────
def handle_jsonrpc_request(request: Dict[str, Any]) -> Dict[str, Any]:
    """JSON-RPC 2.0 요청을 처리합니다."""
    jsonrpc = request.get("jsonrpc")
    method = request.get("method")
    request_id = request.get("id")
    params = request.get("params", {})
    
    # JSON-RPC 2.0 검증
    if jsonrpc != "2.0":
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "error": {
                "code": -32600,
                "message": "Invalid Request: jsonrpc must be '2.0'"
            }
        }
    
    # 메서드별 처리
    try:
        if method == "initialize":
            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "result": {
                    "protocolVersion": "2024-11-05",
                    "capabilities": {
                        "tools": {}
                    },
                    "serverInfo": {
                        "name": "math-server",
                        "version": "1.0.0"
                    }
                }
            }
        
        elif method == "tools/list":
            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "result": {
                    "tools": [
                        {
                            "name": "math",
                            "description": "수학 표현식을 계산합니다. 예: (3 + 5) * 12",
                            "inputSchema": {
                                "type": "object",
                                "properties": {
                                    "expression": {
                                        "type": "string",
                                        "description": "계산할 수학 표현식"
                                    }
                                },
                                "required": ["expression"]
                            }
                        }
                    ]
                }
            }
        
        elif method == "tools/call":
            tool_name = params.get("name")
            arguments = params.get("arguments", {})
            
            if tool_name == "math":
                expression = arguments.get("expression", "")
                result = compute_math(expression)
                return {
                    "jsonrpc": "2.0",
                    "id": request_id,
                    "result": {
                        "content": [
                            {
                                "type": "text",
                                "text": f"계산 결과: {result}"
                            }
                        ]
                    }
                }
            else:
                return {
                    "jsonrpc": "2.0",
                    "id": request_id,
                    "error": {
                        "code": -32601,
                        "message": f"Unknown tool: {tool_name}"
                    }
                }
        
        else:
            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "error": {
                    "code": -32601,
                    "message": f"Method not found: {method}"
                }
            }
    
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "error": {
                "code": -32603,
                "message": f"Internal error: {str(e)}"
            }
        }

# ─── Main Loop ───────────────────────────────────────────────────────────────────
def main():
    """stdin에서 JSON-RPC 요청을 읽고 stdout으로 응답을 출력합니다."""
    for line in sys.stdin:
        line = line.strip()
        if not line:
            continue
        
        try:
            request = json.loads(line)
            
            # Notification인 경우 (id가 없는 경우) 응답하지 않음
            if "id" not in request or request.get("id") is None:
                # notifications/initialized 같은 notification은 무시
                continue
            
            response = handle_jsonrpc_request(request)
        except json.JSONDecodeError as e:
            response = {
                "jsonrpc": "2.0",
                "id": None,
                "error": {
                    "code": -32700,
                    "message": f"Parse error: {e}"
                }
            }
        
        sys.stdout.write(json.dumps(response) + "\n")
        sys.stdout.flush()

if __name__ == "__main__":
    main()


## 9. MCP_weather_server.py


이 코드는 MCP 서버(HTTP)를 실행합니다. Colab에서 서버 실행이 제한될 수 있으니 원본 파일을 참고해 로컬에서 실행하세요.


MCP 날씨 도구 서버(HTTP)를 정의합니다.


In [ ]:
#!/usr/bin/env python3
"""
MCP (Model Context Protocol) Weather Server - HTTP 버전
JSON-RPC 2.0 프로토콜을 사용하여 날씨 정보를 제공합니다.
"""
from fastapi import FastAPI, Request, HTTPException
from pydantic import BaseModel
from typing import Dict, Any, List, Optional, Union
import uvicorn

# ─── JSON-RPC 2.0 Schemas ───────────────────────────────────────────────────
class JSONRPCRequest(BaseModel):
    jsonrpc: str
    method: str
    id: Optional[Union[str, int]] = None
    params: Optional[Dict[str, Any]] = None

class JSONRPCResponse(BaseModel):
    jsonrpc: str
    id: Optional[Union[str, int]]
    result: Optional[Dict[str, Any]] = None
    error: Optional[Dict[str, Any]] = None

# ─── FastAPI App ────────────────────────────────────────────────────────────
app = FastAPI(title="Weather MCP Server")

def get_weather_data(location: str) -> str:
    """더미 날씨 데이터를 반환합니다."""
    location_lower = location.lower().strip()
    
    # 더미 날씨 데이터
    weather_data = {
        "nyc": "뉴욕의 현재 날씨: 화씨 58도 (섭씨 14도), 맑음",
        "new york": "뉴욕의 현재 날씨: 화씨 58도 (섭씨 14도), 맑음",
        "london": "런던의 현재 날씨: 화씨 48도 (섭씨 9도), 흐림",
        "san francisco": "샌프란시스코의 현재 날씨: 화씨 62도 (섭씨 17도), 안개",
        "seoul": "서울의 현재 날씨: 화씨 45도 (섭씨 7도), 맑음",
    }
    
    # 부분 매칭 시도
    for key, value in weather_data.items():
        if key in location_lower or location_lower in key:
            return value
    
    return f"{location}의 날씨 정보를 찾을 수 없습니다. 대략 화씨 65도 (섭씨 18도)입니다."

def handle_jsonrpc_request(request: Dict[str, Any]) -> Dict[str, Any]:
    """JSON-RPC 2.0 요청을 처리합니다."""
    jsonrpc = request.get("jsonrpc")
    method = request.get("method")
    request_id = request.get("id")
    params = request.get("params", {})
    
    # JSON-RPC 2.0 검증
    if jsonrpc != "2.0":
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "error": {
                "code": -32600,
                "message": "Invalid Request: jsonrpc must be '2.0'"
            }
        }
    
    # 메서드별 처리
    try:
        if method == "initialize":
            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "result": {
                    "protocolVersion": "2024-11-05",
                    "capabilities": {
                        "tools": {}
                    },
                    "serverInfo": {
                        "name": "weather-server",
                        "version": "1.0.0"
                    }
                }
            }
        
        elif method == "tools/list":
            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "result": {
                    "tools": [
                        {
                            "name": "weather",
                            "description": "특정 위치의 날씨 정보를 제공합니다. 예: NYC, London, Seoul",
                            "inputSchema": {
                                "type": "object",
                                "properties": {
                                    "location": {
                                        "type": "string",
                                        "description": "날씨를 조회할 위치"
                                    }
                                },
                                "required": ["location"]
                            }
                        }
                    ]
                }
            }
        
        elif method == "tools/call":
            tool_name = params.get("name")
            arguments = params.get("arguments", {})
            
            if tool_name == "weather":
                location = arguments.get("location", "")
                if not location:
                    # 쿼리에서 위치 추출 시도
                    query = arguments.get("query", "")
                    if "weather in" in query.lower():
                        location = query.lower().split("weather in")[1].strip().rstrip("?").strip()
                    else:
                        location = query
                
                weather_info = get_weather_data(location)
                return {
                    "jsonrpc": "2.0",
                    "id": request_id,
                    "result": {
                        "content": [
                            {
                                "type": "text",
                                "text": weather_info
                            }
                        ]
                    }
                }
            else:
                return {
                    "jsonrpc": "2.0",
                    "id": request_id,
                    "error": {
                        "code": -32601,
                        "message": f"Unknown tool: {tool_name}"
                    }
                }
        
        elif method == "notifications/initialized":
            # Notification은 응답하지 않음
            return None
        
        else:
            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "error": {
                    "code": -32601,
                    "message": f"Method not found: {method}"
                }
            }
    
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "error": {
                "code": -32603,
                "message": f"Internal error: {str(e)}"
            }
        }

@app.post("/mcp")
async def handle_mcp(request: Request):
    """
    MCP JSON-RPC 2.0 요청을 처리합니다.
    """
    try:
        body = await request.json()
        
        # Notification인 경우 (id가 없는 경우) 응답하지 않음
        if "id" not in body or body.get("id") is None:
            return {"jsonrpc": "2.0"}
        
        response = handle_jsonrpc_request(body)
        
        if response is None:
            return {"jsonrpc": "2.0"}
        
        return response
        
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "id": None,
            "error": {
                "code": -32700,
                "message": f"Parse error: {str(e)}"
            }
        }

@app.get("/")
async def root():
    """서버 상태 확인"""
    return {
        "status": "running",
        "server": "MCP Weather Server",
        "version": "1.0.0",
        "protocol": "JSON-RPC 2.0"
    }

if __name__ == "__main__":
    # Run with: python3 MCP_weather_server.py
    print("Starting MCP Weather Server on http://0.0.0.0:8000")
    print("MCP endpoint: http://0.0.0.0:8000/mcp")
    uvicorn.run(app, host="0.0.0.0", port=8000)
